This notebook is part of the `kikuchipy` documentation https://kikuchipy.org.
Links to the documentation won't work from the notebook.

# Pattern matching

Import libraries and load the experimental Nickel test data

In [ ]:
# exchange inline for qt5 for interactive plotting from the pyqt package
%matplotlib inline

import matplotlib.pyplot as plt
plt.rcParams["font.size"] = 15
import hyperspy.api as hs
import numpy as np
from orix import sampling
import kikuchipy as kp


s = kp.data.nickel_ebsd_small()  # Use kp.load("data.h5") to load your own data
s

Increase the signal-noise-ratio by removing the diffuse background resulting
from the inelastically scattered electrons

In [ ]:
s.remove_static_background()
s.remove_dynamic_background()

Load the Nickel test master pattern in the Lambert projection

In [ ]:
mp = kp.data.nickel_ebsd_master_pattern_small(projection="lambert")
mp

Get the Nickel phase from the master pattern

In [ ]:
ni = mp.phase
ni

Sample the orientation space of Nickel

In [ ]:
r = sampling.get_sample_fundamental(
    resolution=5, space_group=ni.space_group.number
)
r

Describe the detector dimensions and the detector-sample geometry

In [ ]:
detector = kp.detectors.EBSDDetector(
    shape=s.axes_manager.signal_shape[::-1],
    pc=[0.421, 0.7794, 0.5049],
    sample_tilt=70,
    convention="tsl",
)
detector

Create a dictionary of dynamically simulated EBSD patterns with 20 keV energy

In [ ]:
sim = mp.get_patterns(rotations=r, detector=detector, energy=20)
sim

Match the simulated patterns to the experimental patterns using zero-mean
normalized cross correlation (NCC)

In [ ]:
xmap = s.match_patterns(sim, n_slices=10, keep_n=10)

Get a map of the NCC coefficients of the best matching patterns

In [ ]:
ncc = xmap.get_map_data(xmap.scores[:, 0])
ncc

Get the best matching simulated patterns from the dictionary

In [ ]:
best_patterns = sim.data[xmap.simulation_indices[:, 0]].reshape(s.data.shape)
s_sim = kp.signals.EBSD(best_patterns.compute())
s_sim

Inspect the experimental and best matching simulated patterns in a best scores
map (NCC coefficients)

In [ ]:
hs.plot.plot_signals([s, s_sim], navigator=hs.signals.Signal2D(ncc))